In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, dot, Activation, concatenate, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [ ]:
df = pd.read_csv('/content/dataset.csv')

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
df['english_sentence'] = df['english_sentence'].astype(str).fillna('')
df['hindi_sentence'] = df['hindi_sentence'].astype(str).fillna('')

In [ ]:
english_sentences = df['english_sentence'].tolist()
hindi_sentences = df['hindi_sentence'].tolist()

In [ ]:

eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_sentences)
eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)
max_eng_len = max(len(seq) for seq in eng_sequences)
eng_padded = pad_sequences(eng_sequences, padding='post', maxlen=max_eng_len)



In [ ]:
eng_padded.shape

In [ ]:
hin_tokenizer = Tokenizer()
hin_tokenizer.fit_on_texts(hindi_sentences)
hin_sequences = hin_tokenizer.texts_to_sequences(hindi_sentences)
max_hin_len = max(len(seq) for seq in hin_sequences)
hin_padded = pad_sequences(hin_sequences, padding='post', maxlen=max_hin_len)

In [ ]:
max_eng_len

In [ ]:
max_hin_len

In [ ]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
hin_vocab_size = len(hin_tokenizer.word_index) + 1


In [ ]:
eng_vocab_size

In [ ]:
#model
encoder_inputs = Input(shape=(max_eng_len,))
encoder_embedding = Embedding(input_dim=eng_vocab_size, output_dim=100, input_length=max_eng_len, mask_zero=True)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(256, return_state=True, return_sequences=True)(encoder_embedding)
encoder_states = [state_h, state_c]


In [ ]:
encoder_embedding.shape

In [ ]:
def attention_layer(encoder_outputs, decoder_outputs):
    # Dot product between decoder outputs and encoder outputs
    attention = dot([decoder_outputs, encoder_outputs], axes=[2, 2])
    attention = Activation('softmax')(attention)
    context = dot([attention, encoder_outputs], axes=[2, 1])
    return context


In [ ]:
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=hin_vocab_size, output_dim=100, mask_zero=True)(decoder_inputs)
decoder_lstm, _, _ = LSTM(256, return_sequences=True, return_state=True)(decoder_embedding, initial_state=encoder_states)

In [ ]:
context = attention_layer(encoder_lstm, decoder_lstm)

In [ ]:
decoder_combined_context = concatenate([context, decoder_lstm])

In [ ]:
decoder_dense = TimeDistributed(Dense(hin_vocab_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_combined_context)

In [ ]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


In [ ]:
# Shift target sequences for teacher forcing
hin_sequences_input =hin_padded[:, :-1]
hin_sequences_output = hin_padded[:, 1:]

# Expand dimensions to fit sparse categorical cross-entropy loss
hin_sequences_output = np.expand_dims(hin_sequences_output, -1)


In [ ]:
eng_padded = np.array(eng_padded)
hin_sequences_input = np.array(hin_sequences_input)
hin_sequences_output = np.array(hin_sequences_output)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
eng_train, eng_val, hin_input_train, hin_input_val, hin_output_train, hin_output_val = train_test_split(
    eng_padded, hin_sequences_input, hin_sequences_output, test_size=0.2, random_state=42
)


In [ ]:
# Train the model
history = model.fit(
    [eng_train, hin_input_train],
    hin_output_train,
    batch_size=64,
    epochs=10,
    validation_data=([eng_val, hin_input_val], hin_output_val)
)

